In [ ]:
import json
import pandas as pd
import sqlalchemy as sa

def connect_to_db(config_file="database_creds.json"):
    """Łączy się z bazą danych PostgreSQL za pośrednictwem danych zawartych w pliku database_creds.json."""
    %load_ext sql

    with open(config_file, encoding="utf-8") as db_con_file:
        creds = json.load(db_con_file)

    connection_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{db_name}".format(
        user=creds['user_name'],
        password=creds['password'],
        host=creds['host_name'],
        port=creds['port_number'],
        db_name=creds['db_name']
    )
    
    print("Połączono z bazą.")
    %sql $connection_string

    return connection_string

def import_table_as_csv(connection_string, table):
    """
    Przepisuje zawartość tabeli z bazy danych PostgreSQL do pliku CSV.
    """
    engine = sa.create_engine(connection_string)

    query = f"SELECT * FROM {table}"
    df = pd.read_sql(query, engine)

    csv_file = table + ".csv"
    df.to_csv(csv_file, index=False, encoding='utf-8')

    print(f"Wyeksportowano dane z tabeli '{table}' do pliku '{csv_file}'.")
    return df.head()

def csv_to_table(connection_string, table, csv_file):
    """
    Prepisuje zawartość pliku JSON do tabeli w bazie SQLite.
    Jeśli tabela o podanej nazwie już istnieje, zostanie zamieniona.
    """
    df = pd.read_csv(csv_file, dtype={"PESEL": str})  # Zastosowanie typu string ma na celu zachowanie zer występujących na początku.

    engine = sa.create_engine(connection_string)

    df.to_sql(table, con=engine, if_exists='replace', index=False)

    print(f"Zaimportowano dane do tabeli '{table}'.")